(**Click the icon below to open this notebook in Colab**)

[![Open InColab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiangshiyin/machine-learning-for-actuarial-science/blob/main/2025-spring/week07/notebook/demo.ipynb)

## Time Series data - Example: S&P 500 Index

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt

stock_symbol = '^GSPC'
start_date = '2020-01-01'
end_date = '2024-12-31'

# Download historical data
df_sp500 = yf.download(stock_symbol, start=start_date, end=end_date)
df_sp500.head(10)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(df_sp500['Close'], label='S&P 500')
plt.ylabel('Price')
plt.xlabel('Date')
plt.legend(loc='upper left')
plt.show()


## Time Sereies Decomposition

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

seasonal_decompose(df_sp500.Close.loc['2024-01-01':'2024-12-31'], period=5).plot()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

decomposition = seasonal_decompose(df_sp500.Close.loc['2024-01-01':'2024-12-31'], period=5)
plt.plot(decomposition.resid, label='Residual')
plt.legend()
plt.show()

### Trend

The autocorrelation function (ACF) is a statistical tool to analyze the relationship between the current value of a time series and its lagged values.

$$
ACF(h) = \frac{Cov(y_t, y_{t-h})}{Var(y_t)} = \frac{\sum_{t=h+1}^{T} (y_t - \bar{y})(y_{t-h} - \bar{y})}{\sum_{t=1}^{T} (y_t - \bar{y})^2}
$$

where:
- $T$ is the total number of observations.
- $y_t$ is the value of the time series at time $t$.
- $\bar{y}$ is the mean of the time series.
- $h$ is the lag.

Strong correlations in the ACF plot indicate that the time series has:
- **Significant memory** (dependence on past values).
- **Trends** (slowly decaying correlations).
- **Seasonality** (periodic spikes at regular intervals).

In [ ]:
# plot autocorrelation function(ACF)

sm.graphics.tsa.plot_acf(df_sp500.Close.loc['2024-01-01':'2024-12-31'], lags=30)
plt.xlabel('Number of Lags')
plt.ylabel('ACF')
plt.show()

The partial autocrelation plot (PACF) is a graphical method used to determine the order of an autoregressive (AR) model. It helps identify the significant lagged values that contribute to the behavior of the time series.

$$
PACF(h) = \frac{Cov(y_t, y_{t-h}|y_{t-1},y_{t-2},...,y_{t-h+1})}{\sqrt{Var(y_t|y_{t-1},y_{t-2},...,y_{t-h+1})} \sqrt{Var(y_{t-h}|y_{t-1},y_{t-2},...,y_{t-h+1})}}
$$

Where:
- $y_t$ is the time series at time $t$.
- $h$ is the lag.

In [ ]:
# plot partial autocorrelation function (PACF)

sm.graphics.tsa.plot_pacf(df_sp500.Close.loc['2024-01-01':'2024-12-31'], lags=30)
plt.xlabel('Number of Lags')
plt.ylabel('PACF')
plt.show()

### Seasonality

#### S&P 500

In [ ]:
## plot YOY trend

df2 = df_sp500.Close.copy()
df2.columns = ['GSPC']
df2['Year'] = df2.index.year
df2['DayOfYear'] = df2.index.dayofyear
df2.head(3)

In [ ]:
df2_yoy = df2.pivot(index='DayOfYear', columns='Year', values='GSPC')
df2_yoy.head(3)

In [ ]:
plt.figure(figsize=(12, 6))
for year in df2_yoy.columns:
    plt.plot(df2_yoy.index, df2_yoy[year], label=year)
plt.legend()
plt.title('YoY Trend of SP500 Index')
plt.xlabel('Day of Year')
plt.ylabel('SP500 Index')
plt.grid()
plt.show()

In [ ]:
df3 = df_sp500.Close.copy().rename(columns={'^GSPC': 'GSPC'})
df3['Year'] = df3.index.year
df3['Week'] = df3.index.isocalendar().week
df3.head(3)

In [ ]:
df3_weekly = df3.groupby(['Year', 'Week'])['GSPC'].mean().reset_index()
df3_weekly.head(3)

In [ ]:
df3_weekly_yoy = df3_weekly.pivot(index='Week', columns='Year', values='GSPC')
df3_weekly_yoy.head(3)

In [ ]:
plt.figure(figsize=(12, 6))
for year in df3_weekly_yoy.columns:
    plt.plot(df3_weekly_yoy.index, df3_weekly_yoy[year], label=year)
plt.legend()
plt.title('YoY Trend of SP500 Index')
plt.xlabel('Month of Year')
plt.ylabel('SP500 Index')
plt.grid()
plt.show()

In [ ]:
# plot autocorrelation function(ACF)

sm.graphics.tsa.plot_acf(df3.GSPC, lags=720)
plt.xlabel('Number of Lags')
plt.ylabel('ACF')
plt.show()

In [ ]:
# plot autocorrelation function(ACF)

sm.graphics.tsa.plot_acf(df3_weekly.GSPC, lags=120)
plt.xlabel('Number of Lags')
plt.ylabel('ACF')
plt.show()

#### AAPL

In [ ]:
stock_symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-12-31'

# Download historical data
df_aapl = yf.download(stock_symbol, start=start_date, end=end_date).Close
df_aapl.head(10)

In [ ]:
df_aapl['Year'] = df_aapl.index.year
df_aapl['DayOfYear'] = df_aapl.index.dayofyear

df_aapl.head(3)

In [18]:
df_aapl_yoy = df_aapl.pivot(index='DayOfYear', columns='Year', values='AAPL')

In [ ]:
plt.figure(figsize=(12, 6))
for year in df_aapl_yoy.columns:
    plt.plot(df_aapl_yoy.index, df_aapl_yoy[year], label=year)
plt.legend()
plt.title('YoY Trend of SP500 Index')
plt.xlabel('Day of Year')
plt.ylabel('SP500 Index')
plt.grid()
plt.show()

In [ ]:
df_aapl.head(3)

In [ ]:
# plot autocorrelation function(ACF)

sm.graphics.tsa.plot_acf(df_aapl.AAPL, lags=720)
plt.xlabel('Number of Lags')
plt.ylabel('ACF')
plt.show()

# Classical Methods

In [ ]:
import datetime

ticker = ['AAPL', 'MSFT']
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2021, 1, 1)
stock_prices = yf.download(ticker, start, end, interval='1d').Close

In [ ]:
stock_prices.head(3)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(stock_prices.AAPL, label='AAPL')
plt.plot(stock_prices.MSFT, label='MSFT')
plt.legend()
plt.show()

## Stationary Assumption

Classical time series models typically assume that the underlying data is stationary. Stationarity describes that the time-series has
- Constant mean and mean is not time-dependent
- Constant variance and variance is not time-dependent
- Constant covariance and covariance is not time-dependent

![](https://towardsdatascience.com/wp-content/uploads/2023/04/10vp3L0WZV_HkqS53H6skjg.png)

**[Augmented Dickey-Fuller (ADF) test](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test)**: If the p-value is low (< 0.05), the series is stationary.

In [25]:
from statsmodels.tsa.stattools import adfuller

# ADF Test for stationarity
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    if result[1] < 0.05:
        print("The series is stationary.")
    else:
        print("The series is non-stationary.")

In [ ]:
plt.plot(stock_prices.AAPL, label='AAPL')
plt.axhline(y=stock_prices.AAPL.mean(), color='black', linestyle='--', label='Mean')
plt.legend()
plt.show()

In [ ]:
print(f"Testing the stationarity of the AAPL stock price")
adf_test(stock_prices.AAPL)
print(f"Testing the stationarity of the MSFT stock price")
adf_test(stock_prices.MSFT)

In [ ]:
print(f"Testing the stationarity of the AAPL stock price with difference transformation")
adf_test(stock_prices.AAPL.diff().dropna())
print(f"Testing the stationarity of the MSFT stock price with difference transformation")
adf_test(stock_prices.MSFT.diff().dropna())

In [ ]:
diff_aapl = stock_prices.AAPL.diff().dropna()

plt.plot(diff_aapl, label='AAPL')
plt.axhline(y=diff_aapl.mean(), color='black', linestyle='--', label='Mean')
plt.legend()
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(diff_aapl, lags=30)
plt.xlabel('Number of Lags')
plt.ylabel('ACF')
plt.show()

## Moving Average Model

In [31]:
diff_stock_prices = stock_prices.diff().dropna()
split = int(len(diff_stock_prices['AAPL'].values) * 0.95)
diff_train_aapl = diff_stock_prices['AAPL'].iloc[:split]
diff_test_aapl = diff_stock_prices['AAPL'].iloc[split:]
diff_train_msft = diff_stock_prices['MSFT'].iloc[:split]
diff_test_msft = diff_stock_prices['MSFT'].iloc[split:]

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 6))
plt.tight_layout() 
sm.graphics.tsa.plot_acf(diff_train_aapl,lags=30,
                         ax=ax[0], title='ACF - Apple')
sm.graphics.tsa.plot_acf(diff_train_msft,lags=30,
                         ax=ax[1], title='ACF - Microsoft')
plt.show()

We can see that there are significant spikes at some lags and, therefore, we’ll choose lag 9 for the short MA model and 22 for the long MA for Apple. These imply that an order of 9 will be our short-term order and 22 will be our long-term order in modeling MA

In [ ]:
short_moving_average_appl = diff_train_aapl.rolling(window=9).mean()
long_moving_average_appl = diff_train_aapl.rolling(window=22).mean()

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(diff_train_aapl.loc[start:end].index, 
        diff_train_aapl.loc[start:end],
        label='Stock Price', linestyle='--')
ax.plot(short_moving_average_appl.loc[start:end].index, 
        short_moving_average_appl.loc[start:end],
        label = 'Short MA', linestyle='solid')
ax.plot(long_moving_average_appl.loc[start:end].index, 
        long_moving_average_appl.loc[start:end],
        label = 'Long MA', linestyle='solid')
ax.legend(loc='best')
ax.set_ylabel('Differenced Price')
ax.set_title('Stock Prediction-Apple')
plt.show()

The short-term MA tends to be more responsive to daily changes in Apple’s stock price compared to the long-term MA. This makes sense because taking into account a long MA generates smoother predictions.

In [ ]:
short_moving_average_msft = diff_train_msft.rolling(window=2).mean()
long_moving_average_msft = diff_train_msft.rolling(window=22).mean()

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(diff_train_msft.loc[start:end].index,
        diff_train_msft.loc[start:end],
        label='Stock Price', linestyle='--')
ax.plot(short_moving_average_msft.loc[start:end].index,
        short_moving_average_msft.loc[start:end],
        label = 'Short MA', linestyle='solid')
ax.plot(long_moving_average_msft.loc[start:end].index,
        long_moving_average_msft.loc[start:end],
        label = 'Long MA', linestyle='-.')
ax.legend(loc='best')
ax.set_ylabel('Differenced Price')
ax.set_xlabel('Date')
ax.set_title('Stock Prediction-Microsoft')
plt.show()

## Autoregressive Model

In [ ]:
sm.graphics.tsa.plot_pacf(diff_train_aapl, lags=30)
plt.title('PACF of Apple')
plt.xlabel('Number of Lags')
plt.show()

In [ ]:
sm.graphics.tsa.plot_pacf(diff_train_msft, lags=30)
plt.title('PACF of Microsoft')
plt.xlabel('Number of Lags')
plt.show()

In [37]:
from statsmodels.tsa.ar_model import AutoReg

ar_aapl = AutoReg(diff_train_aapl.values, lags=29)
ar_fitted_aapl = ar_aapl.fit()

In [38]:
ar_predictions_aapl = ar_fitted_aapl.predict(start=len(diff_train_aapl), 
                                             end=len(diff_train_aapl)\
                                             + len(diff_test_aapl) - 1, 
                                             dynamic=False)

In [ ]:
for i in range(len(ar_predictions_aapl)):
    print('==' * 25)
    print('predicted values:{:.4f} & actual values:{:.4f}'\
          .format(ar_predictions_aapl[i], diff_test_aapl[i]))

In [40]:
import pandas as pd

ar_predictions_aapl = pd.DataFrame(ar_predictions_aapl)
ar_predictions_aapl.index = diff_test_aapl.index

In [41]:
ar_msft = AutoReg(diff_train_msft.values, lags=26)
ar_fitted_msft = ar_msft.fit()

ar_predictions_msft = ar_fitted_msft.predict(start=len(diff_train_msft), 
                                             end=len(diff_train_msft)\
                                             +len(diff_test_msft) - 1,
                                             dynamic=False)

ar_predictions_msft = pd.DataFrame(ar_predictions_msft)
ar_predictions_msft.index = diff_test_msft.index

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(18, 15))
 
ax[0].plot(diff_test_aapl, label='Actual Stock Price', linestyle='--')
ax[0].plot(ar_predictions_aapl, linestyle='solid', label="Prediction")
ax[0].set_title('Predicted Stock Price-Apple')
ax[0].legend(loc='best')
ax[1].plot(diff_test_msft, label='Actual Stock Price', linestyle='--')
ax[1].plot(ar_predictions_msft, linestyle='solid', label="Prediction")
ax[1].set_title('Predicted Stock Price-Microsoft')
ax[1].legend(loc='best')
for ax in ax.flat:
    ax.set(xlabel='Date', ylabel='Differenced Price')
plt.show()

## Autoregressive Integrated Moving Average Model

In [ ]:
adf_test(stock_prices.AAPL)

In [ ]:
adf_test(stock_prices.MSFT)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

aapl = stock_prices['AAPL']
size_test = int(len(aapl) * 0.05)
print(size_test)

train_aapl = aapl[:-size_test]
test_aapl = aapl[-size_test:]
diff_aapl_full = aapl.diff().dropna()
diff_train_aapl = train_aapl.diff().dropna()

arima_aapl = ARIMA(train_aapl,order=(9, 1, 9))
arima_fit_aapl = arima_aapl.fit()
arima_pred_aapl = arima_fit_aapl.forecast(steps=size_test)
arima_pred_aapl.index = test_aapl.index

fig, ax = plt.subplots(figsize=(18, 15))
ax.plot(aapl[-size_test:], label='Actual Stock Price', linestyle='--')
ax.plot(arima_pred_aapl, linestyle='solid', label="Prediction")
ax.set_title('Predicted Stock Price-Apple')
ax.legend(loc='best')
ax.set(xlabel='Date', ylabel='Differenced Price')
plt.show()

In [ ]:
test_aapl.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 15))
ax.plot(aapl, label='Actual Stock Price', linestyle='--')
ax.plot(pd.concat([train_aapl, arima_pred_aapl],axis=0), linestyle='solid', label="Prediction")
ax.set_title('Predicted Stock Price-Apple')
ax.legend(loc='best')
ax.set(xlabel='Date', ylabel='Differenced Price')
plt.show()

In [50]:
# import itertools
# p = q = range(0, 9)
# d = range(0, 3)
# pdq = list(itertools.product(p, d, q))
# arima_results_aapl = []
# for param_set in pdq:
#     try:
#         arima_aapl = ARIMA(diff_train_aapl, order=param_set)
#         arima_fitted_aapl = arima_aapl.fit()
#         arima_results_aapl.append(arima_fitted_aapl.aic)
#     except:
#         continue
# print('**'*25)
# print('The Lowest AIC score is {:.4f} and the corresponding parameters are {}'
#       .format(pd.DataFrame(arima_results_aapl)
#              .where(pd.DataFrame(arima_results_aapl).T.notnull().all()).min()[0], 
#              pdq[arima_results_aapl.index(min(arima_results_aapl))]))

In [ ]:
arima_fit_aapl.summary()

AIC vs. BIC

## **1️⃣ Akaike Information Criterion (AIC)**  
$$
AIC = -2 ln(L) + 2k
$$
Where:  
- L = Maximum likelihood of the model  
- k = Number of estimated parameters in the model  
- -2 ln(L) = Measures model fit (lower is better)  
- 2k = Penalizes complexity (more parameters increase AIC)  

---

## **2️⃣ Bayesian Information Criterion (BIC)**  
$$
BIC = -2 ln(L) + k ln(n)
$$
Where:  
- n = Number of observations (sample size)  
- k = Number of parameters  
- ln(n) grows with larger samples → **Stronger penalty for complexity**  

---

## **3️⃣ Comparison of AIC vs. BIC**
| Criterion | Formula | Penalization for Complexity |
|-----------|---------|---------------------------|
| **AIC** | -2 ln(L) + 2k | **Less strict** (penalty = 2k) |
| **BIC** | -2 ln(L) + k ln(n) | **Stronger penalty** (depends on sample size n) |

🔹 **AIC prefers models with better fit** but allows more complexity.  
🔹 **BIC prefers simpler models**, especially for larger datasets.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

score_mae = mean_absolute_error(test_aapl, arima_pred_aapl)
score_rmse = np.sqrt(mean_squared_error(test_aapl, arima_pred_aapl))

print("MAE: ", score_mae)
print("RMSE: ", score_rmse)

In [ ]:
train_aapl.head(3)

# Machine Learning Approaches

## Prophet

Facebook's Prophet is an open-source time series forecasting tool designed for automatic forecasting with minimal manual tuning. It is particularly useful for business and economic time series data with strong seasonality and holiday effects.

In [ ]:
df_train_aapl = train_aapl.reset_index().rename(columns={'Date': 'ds', 'AAPL': 'y'})
df_train_aapl.head(5)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

from prophet import Prophet


# Train the model
model = Prophet()
model.fit(df_train_aapl)

size = 37
x_valid = model.make_future_dataframe(periods=size, freq='d')

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
df_test_aapl = test_aapl.reset_index().rename(columns={'Date': 'ds', 'AAPL': 'y'})
df_y_pred = y_pred.tail(size)[['ds', 'yhat']]
df_combo = pd.merge(df_test_aapl, df_y_pred, on='ds', how='inner')

score_mae = mean_absolute_error(df_combo.y, df_combo.yhat)
score_rmse = math.sqrt(mean_squared_error(df_combo.y, df_combo.yhat))

print("MAE: ", score_mae)
print("RMSE: ", score_rmse)

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
plt.plot(test_aapl, label="Test set", color="orange")
plt.legend()
plt.show()

## From a `time series` problem to a `supervised` problem



In [ ]:
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
print(df)

In [ ]:
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t-1'] = df['t'].shift(1)
print(df)

In [ ]:

from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t+1'] = df['t'].shift(-1)
print(df)

In [58]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
values = [x for x in range(10)]
data = series_to_supervised(values)
print(data)

In [ ]:
values = [x for x in range(10)]
data = series_to_supervised(values, n_in=3)
print(data)

In [ ]:
values = [x for x in range(10)]
data = series_to_supervised(values, n_out=3)
print(data)

### Example: AAPL - Random Forest

In [ ]:
aapl.head(3)

In [ ]:
data = aapl.reset_index().rename(columns={'Date':'ds', 'AAPL':'y'})
data = data.set_index('ds')
data.head(3)

In [64]:
# x = data['y'].shift(6).rolling(window=19).mean().dropna()
# x.head(5)

In [ ]:
data.shape

In [ ]:
for i in range(6, 25):
    data[f'lag_{i}'] = data['y'].shift(i)

data['hour'] = data.index.hour
data['weekday'] = data.index.weekday
data = data.dropna()

data.head(3)

In [67]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
data.head(3)

In [ ]:
lag_cols = [col for col in data.columns if 'lag' in col]
data['rolling_mean'] = data[lag_cols].mean(axis=1)

# extract out the features and labels into separate variables
y = data['y'].values
data2 = data.drop('y', axis=1)

X = data2.values
feature_names = data2.columns
print('dimension: ', X.shape)
data2.head(3)

In [70]:
def train_test_split(X, y, test_size):
    """Perform train-test split with respect to time series structure."""
    test_index = int(len(X) * (1 - test_size))
    X_train = X[:test_index]
    X_test = X[test_index:]
    y_train = y[:test_index]
    y_test = y[test_index:]
    return X_train, X_test, y_train, y_test

In [71]:
test_size = 0.05
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=200)
rfr.fit(X_train, y_train)
prediction = rfr.predict(X_test)

score_mae = mean_absolute_error(y_test, prediction)
score_rmse = np.sqrt(mean_squared_error(y_test, prediction))

print("MAE: ", score_mae)
print("RMSE: ", score_rmse)

In [73]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

plt.figure(figsize=(10,5))
x = range(prediction.size)
plt.plot(x, prediction, label='prediction', linewidth=2.0)
plt.plot(x, y_test, label='actual', linewidth=2.0)


timeseries_cv = TimeSeriesSplit(n_splits=5)
cv = cross_val_score(rfr, X_train, y_train, 
                        cv=timeseries_cv, scoring='neg_mean_absolute_error')
mae = -1 * cv.mean()
deviation = cv.std()

# hard-coded to be 95% confidence interval
scale = 1.96
margin_error = mae + scale * deviation
lower = prediction - margin_error
upper = prediction + margin_error

fill_alpha = 0.2
fill_color = '#66C2D7'
plt.fill_between(x, lower, upper, color=fill_color, alpha=fill_alpha, label='95% CI')      

# if plot_anomalies:
#     anomalies = np.array([np.nan] * len(y_test))
#     anomalies[y_test < lower] = y_test[y_test < lower]
#     anomalies[y_test > upper] = y_test[y_test > upper]
#     plt.plot(anomalies, 'o', markersize=10, label='Anomalies')

error = mean_absolute_percentage_error(prediction, y_test)
plt.title('Mean absolute percentage error {0:.2f}%'.format(error))
plt.legend(loc='best')
plt.tight_layout()
plt.grid(True)

### Example: AAPL - XGBoost

In [ ]:
importances = rfr.feature_importances_
feature_names = data2.columns
# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
# Sort by importance (highest first)
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_importance_df

In [ ]:
## using Xgboost
import xgboost as xgb

xgb_reg = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimator=20,
    learning_rate=0.1,
    random_state=123,
)
xgb_reg.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb_reg.predict(X_test)
score_mae = mean_absolute_error(y_test, y_pred_xgb)
score_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

print("MAE: ", score_mae)
print("RMSE: ", score_rmse)

In [ ]:
importances_xgb = xgb_reg.feature_importances_
feature_names_xgb = data2.columns
# Create a DataFrame for better visualization
feature_importance_df_xgb = pd.DataFrame({'Feature': feature_names_xgb, 'Importance': importances_xgb})
# Sort by importance (highest first)
feature_importance_df_xgb = feature_importance_df_xgb.sort_values(by='Importance', ascending=False)

feature_importance_df_xgb

In [ ]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

plt.figure(figsize=(10,5))
x = range(prediction.size)
plt.plot(x, prediction, label='prediction', linewidth=2.0)
plt.plot(x, y_test, label='actual', linewidth=2.0)


timeseries_cv = TimeSeriesSplit(n_splits=5)
cv = cross_val_score(xgb_reg, X_train, y_train, 
                        cv=timeseries_cv, scoring='neg_mean_absolute_error')
mae = -1 * cv.mean()
deviation = cv.std()

# hard-coded to be 95% confidence interval
scale = 1.96
margin_error = mae + scale * deviation
lower = prediction - margin_error
upper = prediction + margin_error

fill_alpha = 0.2
fill_color = '#66C2D7'
plt.fill_between(x, lower, upper, color=fill_color, alpha=fill_alpha, label='95% CI')      

# if plot_anomalies:
#     anomalies = np.array([np.nan] * len(y_test))
#     anomalies[y_test < lower] = y_test[y_test < lower]
#     anomalies[y_test > upper] = y_test[y_test > upper]
#     plt.plot(anomalies, 'o', markersize=10, label='Anomalies')

error = mean_absolute_percentage_error(prediction, y_test)
plt.title('Mean absolute percentage error {0:.2f}%'.format(error))
plt.legend(loc='best')
plt.tight_layout()
plt.grid(True)